# The Revenge of Rocchio's Angels

We will be #1 this time

Local Script Dependencies

In [1]:
!python -c "import platform; print(platform.platform()); import sys; print(sys.version)"

Windows-10-10.0.26200-SP0
3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [2]:
from engine import SearchEngine

C:\Users\idoli\PycharmProjects\FinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
engine = SearchEngine()

In [4]:
results = engine.get_top_k("international organized crime")

In [8]:
results[0][0][:300]

'<TEXT>\nLanguage: <F P=105> Russian </F>\nArticle Type:BFN\n\n<F P=106> [Feature comprising address to Russia\'s citizens from the </F>\nRussian Ministry of Internal Affairs and "Materials from"\n"Report by the Ministry of Internal Affairs to Russia\'s\nCitizens" under the "ROSSIYSKAYA GAZETA Exclusive" rubr'